In [2]:
import pandas as pd
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import re
import spacy
import string

# Initialize the VADER sentiment analyzer
nltk.download('vader_lexicon')  # Download the VADER lexicon
analyzer = SentimentIntensityAnalyzer()

# Download the NLTK stop words list (if you haven't already)
nltk.download('stopwords')
nltk.download('wordnet')


###################################################################################

#Import Tickets Database
df = pd.read_csv("C:/Users/KevinRomero/OneDrive - CIPPlanner Corporation/Desktop/text analisis Tickets/Support Ticket Customer Unresponsive Days.csv", encoding='latin-1',header=0)

# Replace spaces with underscores in column names
df.columns = df.columns.str.replace(' ', '_')

df = df.drop(columns=['IssueAutoID','AutoCloseFor7Days','Customer_Unresponsive_Days','Status_Date','Customer_Comment_Date','SysDate','URL'])

###################################################################################

#library that contains punctuation

signos_puntuacion = string.punctuation
signos_puntuacion = signos_puntuacion.replace("#", "")  # quitar '#' de hashtags 
signos_puntuacion

#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in signos_puntuacion])
    return punctuationfree

#storing the puntuation free text
df['claned_title'] = df['Title'].apply(lambda x:remove_punctuation(x))

# cambiar los tweets a letras minúsculas
df['claned_title'] = df['claned_title'].apply((lambda x: x.lower()))

###################################################################################

# Function to apply sentiment analysis
def analyze_sentiment(text):
    sentiment_scores = analyzer.polarity_scores(text)
    return sentiment_scores

# Apply sentiment analysis to the 'text_column' and store the results in a new column
df['sentiment_scores'] = df['claned_title'].apply(analyze_sentiment)

# Extract compound scores as a measure of overall sentiment
df['compound_score'] = df['sentiment_scores'].apply(lambda x: x['compound'])

###################################################################################
# Define a function to lemmatize the text in your DataFrame:

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(lemmatized_words)

# Apply the lemmatize_text function to the 'text_column' and store the results in a new column
df['lemmatized_text'] = df['claned_title'].apply(lemmatize_text)
###################################################################################

# Define a function that removes stop words from a given text:
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    words = text.split()
    filtered_text = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_text)

# Apply the remove_stopwords function to the text column in your DataFrame to create a new column with stop words removed:
# Apply the remove_stopwords function to the 'text_column' and store the results in a new column
df['text_without_stopwords'] = df['lemmatized_text'].apply(remove_stopwords)

###################################################################################

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Function to extract noun chunks
def extract_noun_chunks(text):
    doc = nlp(text)
    noun_chunks = [chunk.text for chunk in doc.noun_chunks]
    return noun_chunks

# Apply the extract_noun_chunks function to the 'text_column'
df['noun_chunks'] = df['lemmatized_text'].apply(extract_noun_chunks)

###################################################################################

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Function to extract multi-word noun chunks
def extract_multi_word_noun_chunks(text):
    doc = nlp(text)
    multi_word_chunks = [chunk.text for chunk in doc.noun_chunks if len(chunk.text.split()) > 1]
    return multi_word_chunks



# Apply the extract_multi_word_noun_chunks function to the 'text_column'
df['multi_word_noun_chunks'] = df['lemmatized_text'].apply(extract_multi_word_noun_chunks)

###################################################################################

# Named Entity Recognition (NER) is a natural language processing task that involves identifying and classifying named entities (such as names of people, places, organizations, dates, and more) in text.


# Load the English language model
nlp = spacy.load("en_core_web_sm")

#Define a function to perform NER on a given text:
def perform_ner(text):
    doc = nlp(text)
    named_entities = [(ent.text, ent.label_) for ent in doc.ents]
    return named_entities

# Apply the perform_ner function to a column in your Pandas DataFrame:

# Apply the perform_ner function to the 'text_column'
df['named_entities'] = df['lemmatized_text'].apply(perform_ner)

###################################################################################

# Display the DataFrame
df
###################################################################################



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\KevinRomero\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KevinRomero\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KevinRomero\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,IssueID,Title,Created_By,Created_On,Issue_Ownership,Support_Status,Customer,claned_title,sentiment_scores,compound_score,lemmatized_text,text_without_stopwords,noun_chunks,multi_word_noun_chunks,named_entities
0,CI_000931,URGENT - TXT14301 project workflow issue for P...,Tim Lei,11/6/2013 0:14,CIPPlanner,Closed,MARS - MARS Information Services,urgent txt14301 project workflow issue for pe...,"{'neg': 0.0, 'neu': 0.859, 'pos': 0.141, 'comp...",0.2023,urgent txt14301 project workflow issue for pet...,urgent txt14301 project workflow issue petcare...,"[urgent txt14301 project workflow issue, petca...","[urgent txt14301 project workflow issue, petca...",[]
1,CI_000932,approved clear did not get loaded in Peoplesoft,Robert Drozynski,11/7/2013 12:40,CIPPlanner,Closed,FDXG - FedEx Ground Package System Inc,approved clear did not get loaded in peoplesoft,"{'neg': 0.0, 'neu': 0.526, 'pos': 0.474, 'comp...",0.6597,approved clear did not get loaded in peoplesoft,approved clear get loaded peoplesoft,[peoplesoft],[],[]
2,CI_000933,Add the fields to CLEAR approval report in CMS,Stephen Wu,11/26/2013 21:51,CIPPlanner,Closed,FDXG - FedEx Ground Package System Inc,add the fields to clear approval report in cms,"{'neg': 0.0, 'neu': 0.551, 'pos': 0.449, 'comp...",0.6908,add the field to clear approval report in cm,add field clear approval report cm,"[the field, clear approval report, cm]","[the field, clear approval report]",[]
3,CI_000934,CY Request is blank for several projects - Rep...,Alvaro Santos,12/3/2013 4:41,CIPPlanner,Closed,MARS - MARS Information Services,cy request is blank for several projects repo...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,cy request is blank for several project report...,cy request blank several project report cab bu...,"[cy request, several project report cab budget...","[cy request, several project report cab budget...","[(2014, DATE)]"
4,CI_000935,Report Subscription for SmartPost Daily CLEAR ...,Stephen Wu,12/10/2013 16:41,CIPPlanner,Closed,FDXG - FedEx Ground Package System Inc,report subscription for smartpost daily clear ...,"{'neg': 0.0, 'neu': 0.698, 'pos': 0.302, 'comp...",0.3818,report subscription for smartpost daily clear ...,report subscription smartpost daily clear status,"[subscription, smartpost daily clear status]",[smartpost daily clear status],"[(daily, DATE)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4715,CI_005764,BCPS Report Template Changes Needed,Kristy Inselman,11/1/2023 6:29,CIPPlanner,Being Reviewed by CIPPlanner,ADA - Ada County Highway District,bcps report template changes needed,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,bcps report template change needed,bcps report template change needed,"[bcps, template change]",[template change],[]
4716,CI_005765,UI Modification for Browse Implementation Proj...,Tripp Kennedy,11/1/2023 7:40,CIPPlanner,Being Reviewed by CIPPlanner,WILM - City of Wilmington NC,ui modification for browse implementation proj...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,ui modification for browse implementation project,ui modification browse implementation project,"[ui modification, browse implementation project]","[ui modification, browse implementation project]",[]
4717,CI_005766,Delete IPR,Awasthi Gaurav,11/1/2023 7:55,CIPPlanner,Being Reviewed by CIPPlanner,FFX - Fairfax County VA,delete ipr,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,delete ipr,delete ipr,[delete ipr],[delete ipr],[]
4718,CI_005767,Update PO Amount,Awasthi Gaurav,11/1/2023 9:26,CIPPlanner,Being Reviewed by CIPPlanner,FFX - Fairfax County VA,update po amount,"{'neg': 0.643, 'neu': 0.357, 'pos': 0.0, 'comp...",-0.5574,update po amount,update po amount,[po amount],[po amount],[]
